In [1]:
import pdfplumber
import pandas as pd

In [4]:
# Mostrar hasta 100 filas en la salida
pd.set_option('display.max_rows', 100)

# Mostrar hasta 100 columnas (opcional)
pd.set_option('display.max_columns', 100)

# Mostrar hasta 200 caracteres por columna
pd.set_option('display.max_colwidth', 200)

In [2]:
# Define la ruta completa de tu archivo Parquet
ruta_archivo_parquet = r"C:\Users\Pablo\OneDrive\Maestria en Estadistica\17 - Tesis\Script\03-CotizacionOpciones\DataOpciones\datos_todos_los_pdf.parquet"


# Lee el archivo Parquet en un DataFrame
df = pd.read_parquet(ruta_archivo_parquet)

In [34]:
# Filtrar Filas con vencimientos de opciones y tasa libre de riesgo
df_filtrado = df[df['Texto'].str.startswith('Vencimiento')]

In [35]:
# Ordenar por fecha
df_ordenado = df_filtrado.sort_values(by='Fecha')

In [36]:
# Eliminar Duplicados
df_sin_duplicados = df_ordenado.drop_duplicates()

In [37]:
# Resetear Indice
df_sin_duplicados = df_sin_duplicados.reset_index(drop=True)

In [ ]:
# Split de la columna Texto usando el guion
df_separado = df_sin_duplicados['Texto'].str.split('-', expand=True)

# Concatenar con el DF previo para mantener la columna fecha
df_final = pd.concat([df_sin_duplicados[['Fecha']].reset_index(drop=True), df_separado], axis=1)

# Nuevos nombres de columna:
df_final.columns = ['Fecha', 'Ejercicio_Vto', 'Fecha_Vto','Dias_Vto', 'Tasa_F']


In [ ]:
# Diccionario de meses en español
meses = {
    'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04',
    'mayo': '05', 'junio': '06', 'julio': '07', 'agosto': '08',
    'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12'
}

# Quitar "Vencimiento " y reemplazar mes
df_final['Ejercicio_Vto'] = df_final['Ejercicio_Vto'].str.replace('Vencimiento ', '', regex=False)

# Reemplazar mes en palabras por número
for mes, numero in meses.items():
    df_final['Ejercicio_Vto'] = df_final['Ejercicio_Vto'].str.replace(mes, numero, case=False, regex=False)


In [48]:
# Quitar " días" de columna Días_vto
df_final['Dias_Vto'] = df_final['Dias_Vto'].str.replace(' días', '', regex=False)

In [ ]:
# Separar en dos columnas por el último espacio
df_final[['Tasa', 'Frecuencia']] = df_final['Tasa_F'].str.rsplit(' ', n=1, expand=True)

# Quitar la columna Tasa_F
df_final = df_final.drop(columns=['Tasa_F'])

In [ ]:
df_final = df_final.rename(columns={'Ejercicio_Vto':'Mes_Vto','Tasa': 'Tasa_Tipo', 'Frecuencia': 'Tasa'})


In [ ]:
# Eliminar caracter % de Tasa
df_final['Tasa'] = df_final['Tasa'].str.replace('%', '', regex=False)

# Convertir el floar la columna Tasa
df_final['Tasa'] = df_final['Tasa'].astype(float)

In [62]:
df_final

,Fecha,Mes_Vto,Fecha_Vto,Dias_Vto,Tasa_Tipo,Tasa
0,2010-01-04,01,15/01/2010,11,Tasa Baibar,9.46
1,2010-01-04,02,19/02/2010,46,Tasa Baibar,9.46
2,2010-01-04,04,16/04/2010,102,Tasa Baibar,9.46
3,2010-01-04,06,18/06/2010,165,Tasa Baibar,9.46
4,2010-01-05,01,15/01/2010,10,Tasa Baibar,8.84
...,...,...,...,...,...,...
13777,2024-04-23,04,26/04/2024,3,Tasa Promedio de Caución a 30 días,63.78
13778,2024-04-23,05,17/05/2024,24,Tasa Promedio de Caución a 30 días,63.78
13779,2024-04-23,06,21/06/2024,59,Tasa Promedio de Caución a 30 días,63.78
13780,2024-04-23,08,16/08/2024,115,Tasa Promedio de Caución a 30 días,63.78


In [61]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13782 entries, 0 to 13781
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Fecha      13782 non-null  object 
 1   Mes_Vto    13782 non-null  object 
 2   Fecha_Vto  13782 non-null  object 
 3   Dias_Vto   13782 non-null  object 
 4   Tasa_Tipo  13782 non-null  object 
 5   Tasa       13782 non-null  float64
dtypes: float64(1), object(5)
memory usage: 646.2+ KB


In [ ]:
df_final['Fecha_Vto'] = pd.to_datetime(df_final['Fecha_Vto'], errors='coerce')
df_final['Fecha'] = pd.to_datetime(df_final['Fecha'], errors='coerce')
df_final['Mes_Vto'] = df_final['Mes_Vto'].astype(int)
df_final['Dias_Vto'] = df_final['Dias_Vto'].astype(int)

C:\Users\Pablo\AppData\Local\Temp\ipykernel_10736\2563263055.py:1: UserWarning: Parsing dates in  %d/%m/%Y  format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_final['Fecha_Vto'] = pd.to_datetime(df_final['Fecha_Vto'], errors='coerce')


In [68]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13782 entries, 0 to 13781
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Fecha      13782 non-null  datetime64[ns]
 1   Mes_Vto    13782 non-null  int32         
 2   Fecha_Vto  13782 non-null  datetime64[ns]
 3   Dias_Vto   13782 non-null  int32         
 4   Tasa_Tipo  13782 non-null  object        
 5   Tasa       13782 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int32(2), object(1)
memory usage: 538.5+ KB


In [69]:
df_final

,Fecha,Mes_Vto,Fecha_Vto,Dias_Vto,Tasa_Tipo,Tasa
0,2010-01-04,1,2010-01-15,11,Tasa Baibar,9.46
1,2010-01-04,2,2010-02-19,46,Tasa Baibar,9.46
2,2010-01-04,4,2010-04-16,102,Tasa Baibar,9.46
3,2010-01-04,6,2010-06-18,165,Tasa Baibar,9.46
4,2010-01-05,1,2010-01-15,10,Tasa Baibar,8.84
...,...,...,...,...,...,...
13777,2024-04-23,4,2024-04-26,3,Tasa Promedio de Caución a 30 días,63.78
13778,2024-04-23,5,2024-05-17,24,Tasa Promedio de Caución a 30 días,63.78
13779,2024-04-23,6,2024-06-21,59,Tasa Promedio de Caución a 30 días,63.78
13780,2024-04-23,8,2024-08-16,115,Tasa Promedio de Caución a 30 días,63.78


In [70]:
frecuencias = df_final['Tasa_Tipo'].value_counts()
print(frecuencias)


Tasa_Tipo
Tasa Baibar                           10713
Tasa Promedio de Caución a 30 días     3069
Name: count, dtype: int64


In [ ]:
# Se reemplaza los valores string a int para ahorrar memoria
# Tasa Baibar                           1
# Tasa Promedio de Caución a 30 días    2

# Eliminar espacios sobrantes
df_final['Tasa_Tipo'] = df_final['Tasa_Tipo'].str.strip()

# Reemplazar por números
reemplazos = {
    'Tasa Baibar': 1,
    'Tasa Promedio de Caución a 30 días': 2
}
df_final['Tasa_Tipo'] = df_final['Tasa_Tipo'].replace(reemplazos)

# Convertir a int solo si todos los valores ya son numéricos
df_final['Tasa_Tipo'] = df_final['Tasa_Tipo'].astype(int)

C:\Users\Pablo\AppData\Local\Temp\ipykernel_10736\3616691393.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final['Tasa_Tipo'] = df_final['Tasa_Tipo'].replace(reemplazos)


In [74]:
df_final

,Fecha,Mes_Vto,Fecha_Vto,Dias_Vto,Tasa_Tipo,Tasa
0,2010-01-04,1,2010-01-15,11,1,9.46
1,2010-01-04,2,2010-02-19,46,1,9.46
2,2010-01-04,4,2010-04-16,102,1,9.46
3,2010-01-04,6,2010-06-18,165,1,9.46
4,2010-01-05,1,2010-01-15,10,1,8.84
...,...,...,...,...,...,...
13777,2024-04-23,4,2024-04-26,3,2,63.78
13778,2024-04-23,5,2024-05-17,24,2,63.78
13779,2024-04-23,6,2024-06-21,59,2,63.78
13780,2024-04-23,8,2024-08-16,115,2,63.78


In [75]:
# Guardar como parquet:
df_final.to_parquet('df_vtos_tasa.parquet', index=False, compression='snappy')
